In [ ]:
# step 1

In [ ]:
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# =============== tool ====================================

prices = {"London": 150, "Paris": 70, "Lisbon": 120, "Berlin": 75}

def ticket_price(city):
    print(f"the price of a ticket to {city} is {prices[city]}")
    return prices[city]

tools = [
    {
        "type": "function",
        "function": {
            "name": "ticket_price",
            "description": "Get ticket prices to travel to a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"}
                },
                "required": ["city"]
            }
        }
    }
]

# =========================== chatbot assistant =============================
model_ = "gpt4"
SYSTEM_PROMPT = "You're an Assitant of a Airport Agency to help user to get some information"


def chat_func(msg, history):
    history = [{"role": h["role"], "content":h["content"]} for h in history]

    message = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": msg}]

    response = client.chat.completions.create(model = model_, messages=message, tools=tools)

    return response.choices[0].message


demo = gr.ChatInterface(fn = chat_func, type='messages')
demo.launch()

### output of step 1

#### Explaination

1. LLM doesn't know the price >> should ask from "ticket_price" function
2. LLM never sees the python function >> sees the function description (a list of a dictionary)
3. user asks "How much is a ticket to Paris?"

    in chat_func a message created:
    message = [
                {"role": "system", "content": "You're an Assistant of an Airport Agency..."},
                {"role": "user", "content": "How much is a ticket to Paris?"}
            ]

4. LLM sees some info "ticket", "price", "Paris" >> call the tool

    model output:
        {
        "role": "assistant",
        "content": null,
        "tool_calls": [
            {
            "id": "call_123",
            "type": "function",
            "function": {
                "name": "ticket_price",
                "arguments": "{\"city\": \"Paris\"}"
            }
            }
        ]
        }

    * content is null since model isn't answering yet

5. Execute the tool >> python runs "result = ticket_price("Paris")"

    outputs: 
    the price of a ticket to Paris is 7070
    70

6. Python sends a message to the model (LLM)

    {
    "role": "tool",
    "tool_call_id": "call_123",
    "name": "ticket_price",
    "content": "70"
    }

7. new message will be a combination of these roles: system, user, assistant, tool

    [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": "How much is a ticket to Paris?"},
        {
            "role": "assistant",
            "content": null,
            "tool_calls": [...]
        },
        {
            "role": "tool",
            "name": "ticket_price",
            "content": "70"
        }
    ]

8. final answer is:

    {
        "role": "assistant",
        "content": "A ticket to Paris costs 70 euros. Let me know if you'd like prices for other cities!"
    }


In [ ]:
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# =============== tool ====================================

prices = {"London": 150, "Paris": 70, "Lisbon": 120, "Berlin": 75}

def ticket_price(city):
    print(f"the price of a ticket to {city} is {prices[city]}")
    return prices[city]

tools = [
    {
        "type": "function",
        "function": {
            "name": "ticket_price",
            "description": "Get ticket prices to travel to a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"}
                },
                "required": ["city"]
            }
        }
    }
]

# =========================== chatbot assistant =============================
model_ = "gpt4"
SYSTEM_PROMPT = "You're an Assitant of a Airport Agency to help user to get some information"


def chat_func(msg, history):
    history = [{"role": h["role"], "content":h["content"]} for h in history]

    message = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": msg}]

    response = client.chat.completions.create(model = model_, messages=message, tools=tools)

    assistant_msg = response.choices[0].message     # LLM isn't answering yet, should detect the tool calling

    if assistant_msg.tool_calls:     # check tool calling, the result is None or a list of tool calls
        tool_call = assistant_msg.tool_calls[0]

        # get the arg from the LLM message and pass it to the python function
        import json

        args = json.loads(tool_call.function.arguments)
        city = args["city"]

        # call the function
        tool_result = ticket_price(city)

        # now we have the price, create the message 
        tool_message = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(tool_result)
        }

        # And append messages
        messages = message + [assistant_msg, tool_message]

        # Second model call (final answer)
        final_response = client.chat.completions.create(
            model=model_,
            messages=messages,
            tools=tools
        )
        
        return final_response.choices[0].message.content
    
    return assistant_msg.content

demo = gr.ChatInterface(fn = chat_func, type='messages')
demo.launch()